In [ ]:
# Data preparation
# Input data  : /OFH-clean-data/HES_APC_ADMISSIONS_DIAGNOSIS.csv, /OFH-clean-data/OFH_Questionnaire_clean.csv, /OFH-clean-data/OFH_Questionnaire_Data_dictionary_clean.csv  
# Output data : /_outputs/participant-data.xlsx/pkl/csv

In [ ]:
# import libraries 

# please try to to import libraries stored in another jupyter notebook
# this notebook is called 'my_libraries'
# and is imported as follows

import import_ipynb
from my_libraries import *

In [ ]:
# if importing via a notebook does not work, please uncomment the below

#import io, os, sys, types
#from IPython import get_ipython
#from nbformat import read
#from IPython.core.interactiveshell import InteractiveShell
#import numpy as np
#import pandas as pd
#from datetime import date
#from scipy import stats
#from scipy.stats import norm
#import seaborn as sns
#import matplotlib
#import matplotlib.pyplot as plt
#import pickle
#from sklearn.linear_model import LogisticRegression
#from sklearn.model_selection import StratifiedShuffleSplit
#from sklearn.metrics import confusion_matrix, accuracy_score
#from sklearn.preprocessing import StandardScaler
#from sklearn.preprocessing import MinMaxScaler
#from sklearn.model_selection import train_test_split
#from sklearn import metrics
#import sklearn.metrics as metrics
#from sklearn.metrics import classification_report
#sns.set_palette("BuPu")

In [ ]:
# set directories 

os.chdir('/Users/bethanyisaacs/Documents/git-new/tre-demo-epi-test/DemoEpidemiologyScenario/')

indir = os.getcwd()+'/OFH-data'

#  make output directory 

if not os.path.exists('_outputs'):
    os.makedirs('_outputs')

outdir = os.getcwd()+'/_outputs'

In [ ]:
# import hes data, and set PID to index 
 
hes =  pd.read_csv(indir+'/HES_APC_ADMISSIONS_DIAGNOSIS.csv')
hes.set_index('pid', inplace=True)

In [ ]:
# import questionnaire data

qx_ = pd.read_csv(indir+'/OFH_Questionnaire_clean.csv')

In [ ]:
qx_.head()

In [ ]:
# import template data 

dict_ = pd.read_csv(indir+'/OFH_Questionnaire_Data_dictionary_clean.csv')

In [ ]:
dict_.head()

In [ ]:
# select columns of interest from questionnaire data based on their hashId

db = 'q-019f4cea-245c-43f4-b0f5-bcb00b1fc8b2' # primary diagnoses 
gn = 'q-33274eb3-0e5c-4101-b2ff-723025d4bb4e' # sex
et = 'q-c732aa9e-4a03-4669-8afc-d301524e267d' # ethnicity 
ht = 'q-7cb272fa-ebd3-4b51-b700-9fa791934008' # height - continious
wt = 'q-337d0852-2674-4432-a2d6-109939b54e26' # weight - continous 
ic = 'q-1a37f6df-f17a-4401-a837-d47e37737215' # income - categorical 
sm = 'q-6283cdbe-2b91-441b-bdb7-e0ed5bd26829' # smoking now - categorical 
wk = 'q-b627223f-1b81-4615-93ad-683fde65cfac' # times walking for pleasure - categorical
oh = 'q-441dd55c-d351-4bc2-87c4-69b418952cff' # overall health - categorical

# rename the sex column to the original key in the questionnaire template for remapping purposes

qx_.rename(columns={'sex' : gn}, inplace=True)

# subset questionnaire data to contain all participants but only the columns listed above (plus year of birth)

cols = [*qx_.columns[qx_.columns.str.endswith((gn, et, ht, wt, ic, sm, wk, oh, db))]] # list of columns to be retained in the dataset
qx_subset = pd.concat([qx_[['pid', 'year of birth']], qx_[cols]], axis=1).set_index('pid') # subsetted dataset

In [ ]:
qx_subset.head()

In [ ]:
# create diabetic_status as a binary variable and save as a csv file 

diabetes_only = qx_subset[['year of birth' , 'q-019f4cea-245c-43f4-b0f5-bcb00b1fc8b2']].copy()
diabetes_only.loc[(diabetes_only['q-019f4cea-245c-43f4-b0f5-bcb00b1fc8b2'] == 6), 'diabetic_status'] = '1' 
diabetes_only = diabetes_only.fillna(0)
diabetes_only = diabetes_only.drop(columns=['year of birth', 'q-019f4cea-245c-43f4-b0f5-bcb00b1fc8b2'])#.diabetic_score.value_counts()
diabetes_only.to_csv(outdir+'/diabetic_status.csv')

In [ ]:
diabetes_only.head()

In [ ]:
# create answer mapping dictionary from the qxdict_ dataframe which contains the questionnaire template information 
# dict_ is a dataframe that contains a column for the Answer mappings. The Answer mappings refer to the numeric value stored in the questionnaire submissions and the categorical variable, per question.
# which are stored in a key-value pairing within a dictionary 

qxdict_ = dict_[dict_['QuestionId'].isin(cols)].dropna(subset=['Answers']) # drop rows in template where the Answer column contains empty dictionaries
qxdict_['Answers'] = qxdict_.apply(lambda x: eval(str(x.Answers)), axis=1) # eval the Answer column so we can properly extract the dictionary fields

qxdict_dic = dict(zip(qxdict_['QuestionId'], qxdict_['Answers'])) # create question - answer dictionary from qxdict_ dataframe
qxdict_dic = {k.strip(): v for (k, v) in qxdict_dic.items()} # drop keys with empty values from dicitonary 

In [ ]:
# map answers in the questionnaire dataset based on the qxdict_dic dictionary 

qx_map = qx_subset.copy()

for k, v in qxdict_dic.items():
    if k in qx_map:
        qx_map[k] = qx_map[k].apply(lambda x: next((d["answer"] for d in v if d["value"] == str(x)), x)) # remapping the value in the submission with the answer field from the Asnwer dictionary 

# create alias column names in the mapped dataframe 

qx_map.rename(columns=dict(zip(cols, ['sex', 'height', 'weight', 'ethnicity', 'income', 'walking', 'smoking', 'general health', 'primary diagnoses'])), inplace= True )

In [ ]:
# select diabetic participants from hes

hstack = hes[[*hes.columns[hes.columns.str.startswith('diag')]]].stack().to_frame().droplevel(level=1) # stack all columns in the hes dataset
dhs = hstack[hstack[0] == 'E10.0'].rename(columns={0: 'ICD code'}) # subset rows that include E10.0 only - which is the code for Diabetes

#   11 diabetic participants who are in both the questionnaire and hes datasets 
#   1 participant has Diabetes as their primary diagnoses 
#   check with the following : qx_map[qx_map.index.isin([*dhs.index])]

In [ ]:
# left join
# merge quesitonnarie and hes datasets on pids 

dat = pd.merge(qx_map, dhs, how = 'outer', left_on=qx_map.index, right_on=dhs.index).rename(columns={'key_0' : 'pid'}).set_index('pid')
dat[dat['ICD code'].notna()]

# factorize diabetic status

dat['diabetic status'] = 0 # add a column with 0 as default value
dat.loc[(dat['ICD code'] == 'E10.0') | (dat['primary diagnoses'] == 'Diabetes'),'diabetic status'] = 1 # map diabetic status to 1

# set source of data
dat['diabetic source'] = dat['ICD code'].map({'E10.0' : 'hes'}) 
dat.loc[(dat['primary diagnoses'] == 'Diabetes'), 'diabetic source'] = 'qx' 
dat.loc[(dat['ICD code'] == 'E10.0') & (dat['primary diagnoses']=='Diabetes'), 'diabetic source'] = 'both' 


In [ ]:
# save output in various formats 

# pickle 

with open(outdir+'/participant-data.pkl', 'wb') as pickle_file:
    pickle.dump([dat], pickle_file)

# csv 

dat.to_csv(outdir+'/participant-data.csv')

# excel

dat.to_excel(outdir+'/participant-data.xlsx')

In [ ]:
dat.head()